## Transfer Learning Inception V3 using Keras

Please download the dataset from the below url

In [46]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1771: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [47]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import tensorflow as tf

In [48]:
!nvidia-smi

Fri Feb  4 05:00:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    60W / 149W |   2324MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [51]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/insect/UPL Assignment/training_images'
valid_path = '/content/drive/MyDrive/insect/UPL Assignment/testing_images'


In [52]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [53]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [54]:
  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/insect/UPL Assignment/training_images/*')

In [55]:
folders

['/content/drive/MyDrive/insect/UPL Assignment/training_images/Stem Borer Larvae',
 '/content/drive/MyDrive/insect/UPL Assignment/training_images/Leaf Folder Larvae',
 '/content/drive/MyDrive/insect/UPL Assignment/training_images/Brown Plant Hopper Nimph',
 '/content/drive/MyDrive/insect/UPL Assignment/training_images/Brown Plant Hopper Adult']

In [56]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [57]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [58]:

# view the structure of the model
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_190 (Conv2D)            (None, 111, 111, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_188 (Batch  (None, 111, 111, 32  96         ['conv2d_190[0][0]']             
 Normalization)                 )                                                           

In [59]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [60]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [61]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/insect/UPL Assignment/training_images',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'input')

Found 314 images belonging to 4 classes.


In [62]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/insect/UPL Assignment/testing_images',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 24 images belonging to 4 classes.


In [63]:
training_data_scal = ImageDataGenerator(rescale=1/255)

In [64]:
tranining_data = training_data_scal.flow_from_directory('/content/drive/MyDrive/insect/UPL Assignment/training_images',
                                                       target_size=(256,256),
                                                       batch_size=10,
                                                      class_mode='categorical')

Found 314 images belonging to 4 classes.


In [65]:
!pip install keras

In [66]:
from tensorflow.keras.optimizers import SGD

In [67]:
#fit_generator(generator, epochs, steps_per_epoch, validation_data, validation_steps)

In [68]:
from tensorflow.keras import layers
model = tf.keras.models.Sequential([layers.Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)),
                                   layers.MaxPool2D(2,2),
                                   layers.Conv2D(16,(3,3),activation='relu'),
                                   layers.MaxPool2D(2,2),
                                   layers.Flatten(),
                                   layers.Dense(512,activation='relu'),
                                   layers.Dense(4,activation='softmax')])

In [69]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_284 (Conv2D)         (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_285 (Conv2D)         (None, 125, 125, 16)      4624      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 62, 62, 16)       0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 61504)             0         
                                                                 
 dense_5 (Dense)             (None, 512)              

In [70]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['acc'])

In [80]:
history=model.fit_generator(tranining_data,
                    epochs=15,
                    steps_per_epoch=10,
                   verbose=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/15
10/10 [==============================] - 19s 2s/step - loss: 0.0579 - acc: 0.9787
Epoch 2/15
10/10 [==============================] - 19s 2s/step - loss: 0.0368 - acc: 0.9900
Epoch 3/15
10/10 [==============================] - 20s 2s/step - loss: 0.0608 - acc: 0.9700
Epoch 4/15
10/10 [==============================] - 18s 2s/step - loss: 0.0918 - acc: 0.9681
Epoch 5/15
10/10 [==============================] - 20s 2s/step - loss: 0.0626 - acc: 0.9900
Epoch 6/15
10/10 [==============================] - 20s 2s/step - loss: 0.0658 - acc: 0.9800
Epoch 7/15
10/10 [==============================] - 20s 2s/step - loss: 0.0539 - acc: 0.9900
Epoch 8/15
10/10 [==============================] - 20s 2s/step - loss: 0.0944 - acc: 0.9800
Epoch 9/15
10/10 [==============================] - 20s 2s/step - loss: 0.1117 - acc: 0.9900
Epoch 10/15
10/10 [==============================] - 18s 2s/step - loss: 0.0540 - acc: 1.0000
Epoch 11/15
10/10 [==============================] - 18s 2s/step - lo

In [79]:
# fit the model
# Run the cell. It will take some time to execute'''
'''
r = model.fit_generator(
  training_data,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=2)
 # validation_data=4)
  # verbose=1 ) 
  #callbacks=None
#)

#validation_steps=len(test_set)
#
'''

'\nr = model.fit_generator(\n  training_data,\n  validation_data=test_set,\n  epochs=10,\n  steps_per_epoch=2)\n # validation_data=4)\n  # verbose=1 ) \n  #callbacks=None\n#)\n\n#validation_steps=len(test_set)\n#\n'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss
'''
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')
'''